This notebook presents a step-by-step tutorial on using the developed packages for price forecasting.

In [ ]:
# Add package folder to Python path
import sys
sys.path.append('./')
import forecasting as fr
import importlib
import pandas as pd
importlib.reload(fr.get_data)
importlib.reload(fr.model)
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
import matplotlib.dates as mdates

from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_absolute_percentage_error

import tensorflow as tf

## Data acquisition

Import the data as a dataframe. The helper function cleans up the data from API and only gives price and volume data.

NG ESO API - The API is *inclusive* of start date and *exclusive* of the end date.

In [ ]:
query_start_date = '2024-06-10'
query_end_date = '2025-06-10'

dcl_data = fr.get_data.get_historical_fr_data_price_volume(query_start_date, query_end_date)

In [ ]:
dcl_data.head()

## Naive forecast

In [ ]:
# Calculate the naive forecast using a rolling average for both price and volume
# Give the data for days -1 (index -7 onwards) to -8 (until index -49) and verify the forecast for day 0.

price_naive_forecast = fr.model.rolling_average_multi_timeperiod(7, 6, dcl_data['clearingPrice'].iloc[:-6])
volume_naive_forecast = fr.model.rolling_average_multi_timeperiod(7, 6, dcl_data['clearedVolume'].iloc[:-6])

# Compare the naive forecast with the actual values for day 0
# Calculate the MAPE
mape_price_naive = mean_absolute_percentage_error(dcl_data['clearingPrice'].iloc[-6:], price_naive_forecast)
mape_volume_naive = mean_absolute_percentage_error(dcl_data['clearedVolume'].iloc[-6:], volume_naive_forecast)

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))

timestamp = dcl_data.index[-6:]

# First the price plot
ax1.plot(timestamp, dcl_data['clearingPrice'].iloc[-6:], marker='o', linestyle='-', color='blue', label='Actual Prices')
ax1.plot(timestamp, price_naive_forecast, marker='o', linestyle='-', color='red', label='Naive Forecast Prices')

ax1.set_xlabel('Delivery Start Time')
ax1.set_ylabel('Clearing Price (£/MWh)')
ax1.set_title('Price comparison. MAPE: {:.2f}%'.format(mape_price_naive * 100))
ax1.legend()

ax1.set_xticks(timestamp) # Shows ticks only at actual data points
ax1.set_xticklabels(timestamp.strftime("%H:%M"), rotation=0) # Shows only the hour and minute
ax1.grid(True)

# Next the volume plot
ax2.plot(timestamp, dcl_data['clearedVolume'].iloc[-6:], marker='o', linestyle='-', color='blue', label='Actual Volume')
ax2.plot(timestamp, volume_naive_forecast, marker='o', linestyle='-', color='red', label='Naive Forecast Volume')

ax2.set_xlabel('Delivery Start Time')
ax2.set_ylabel('Clearing Volume (MWh)')
ax2.set_title('Volume comparison. MAPE: {:.2f}%'.format(mape_volume_naive * 100))
ax2.legend()
ax2.set_xticks(timestamp) # Shows ticks only at actual data points
ax2.set_xticklabels(timestamp.strftime("%H:%M"), rotation=0) # Shows only the hour and minute
ax2.grid(True)


plt.tight_layout()
plt.show()

The naive forecast is... well... naive. It gives an error of over 51% for the prices which is quite rubbish. It's also worth noting from the above figure that the predictions are very much off for the first three time-steps, but quite good for the last three timesteps.

The forecast is quite accurate for volume prediction, however.

Time to use a deep learning model.

## Forecasting using LSTM

Autocorrelation analysis (in a different notebook) suggests that the time-series data has a good correlation with lags of 6, 12, and 18 timesteps. So we use that to generate lagged features for our LSTM model.

In [ ]:
lag_times = [6, 12, 18]

# Create lagged features for the specified lag times
for lag in lag_times:
    dcl_data[f'Price_lag_{lag}'] = dcl_data['clearingPrice'].shift(lag)
    # dcl_data[f'Volume_lag_{lag}'] = dcl_data['clearedVolume'].shift(lag)

# Just one lag for volume
dcl_data[f'Volume_lag_{6}'] = dcl_data['clearedVolume'].shift(6)

print("Lagged features created for both price and volume lags:", lag_times)

# Drop rows that don't have enough history for the lagged features
dcl_data = dcl_data.dropna()

# print("First few rows of the dataframe are: \n", dcl_data.head())
# print("Last few rows of the dataframe are: \n", dcl_data.tail())

In [ ]:
# Split the dataset into training and validation sets
tscv = TimeSeriesSplit(n_splits=2)
for train_index, test_index in tscv.split(dcl_data):
    X_train, X_valid = dcl_data.iloc[train_index], dcl_data.iloc[test_index]

print("Train shape:", X_train.shape, "Validation shape:", X_valid.shape)
print("Original data shape:", dcl_data.shape)

In [ ]:
# Define the LSTM parameters
WINDOW_SIZE = 24
BATCH_SIZE = 32
SHUFFLE_BUFFER_SIZE = 1000

In [ ]:
# Generate the dataset windows that go into the LSTM model

series_train = X_train['clearingPrice'].values
X_lag_train = X_train.drop(columns=['clearingPrice', 'clearedVolume']).values
series_train = tf.convert_to_tensor(series_train, dtype=tf.float32)
X_lag_train = tf.convert_to_tensor(X_lag_train, dtype=tf.float32)

series_valid = X_valid['clearingPrice'].values
X_lag_valid = X_valid.drop(columns=['clearingPrice', 'clearedVolume']).values
series_valid = tf.convert_to_tensor(series_valid, dtype=tf.float32)
X_lag_valid = tf.convert_to_tensor(X_lag_valid, dtype=tf.float32)

train_set = fr.model.windowed_dataset_multi_input(
    series=series_train,
    X_lag_full=X_lag_train,
    window_size=WINDOW_SIZE,
    batch_size=BATCH_SIZE,
    shuffle_buffer=SHUFFLE_BUFFER_SIZE
)

valid_set = fr.model.windowed_dataset_multi_input(
    series=series_valid,
    X_lag_full=X_lag_valid,
    window_size=WINDOW_SIZE,
    batch_size=BATCH_SIZE,
    shuffle_buffer=SHUFFLE_BUFFER_SIZE
)

In [ ]:
# Get the LSTM model
lstm_model = fr.model.get_LSTM_model(
    window_size= WINDOW_SIZE,  
    learning_rate= 1e-6, 
    momentum= 0.9, 
    num_lag_features= dcl_data.shape[1] - 2)
print(lstm_model.summary())

In [ ]:
# Train the model
# Reset states generated by Keras
tf.keras.backend.clear_session()

# Get initial weights
init_weights = lstm_model.get_weights()

# Reset the weights
lstm_model.set_weights(init_weights)

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience= 10,
    restore_best_weights=True
)

# Fit the model
history = lstm_model.fit(
    train_set,
    epochs=100,
    validation_data=valid_set,
    callbacks=[early_stopping],
    verbose=1
)

In [ ]:
# Check if the training was successful by visualising the training history

# Get mae and loss from history log
mae=history.history['mae']
loss=history.history['loss']

# Get number of epochs
epochs=range(len(loss)) 

# Plot mae and loss
fr.get_data.plot_series(
    x=epochs, 
    y=(mae, loss), 
    title='MAE and Loss', 
    xlabel='Epochs',
    legend=['MAE', 'Loss']
    )


In [ ]:
lstm_forecast = fr.model.lstm_forecast_multi_input(
    model=lstm_model,
    series=series_valid,
    X_lag_full=X_lag_valid,
    window_size=WINDOW_SIZE,
    batch_size=BATCH_SIZE
)

In [ ]:
# Calculate the MAPE for the forecast for the last day (6 periods)
mape_forecast = mean_absolute_percentage_error(
    dcl_data['clearingPrice'].iloc[-6:],
    lstm_forecast[-6:]
)  
print(f"MAPE for LSTM forecast: {mape_forecast*100:.2f}%")

print(f"MAPE for naive forecast was: {mape_price_naive*100:.2f}%")

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(timestamp, X_valid['clearingPrice'].iloc[-6:], marker='o', linestyle='-', color='blue', label='Actual Prices')
plt.plot(timestamp, price_naive_forecast, marker='o', linestyle='-', color='red', label='Naive Forecast predictions. MAPE: {:.2f}%'.format(mape_price_naive * 100))
plt.plot(timestamp, lstm_forecast[-6:], marker='o', linestyle='-', color='black', label='LSTM (with lags) predictions. MAPE: {:.2f}%'.format(mape_forecast * 100))
plt.title('Forecast vs actual prices for the last day')
plt.xlabel('Time')
plt.ylabel('Price (£/MWh)')
plt.legend()
plt.grid(True)
# Format x-tick labels as HH:MM
plt.show()